In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/데이터분석캡스톤디자인/CLOVAOCR4')

Mounted at /content/drive


In [ ]:
import sys
import time
import random
import string
import argparse

import torch
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data
import numpy as np

from utils import CTCLabelConverter, CTCLabelConverterForBaiduWarpctc, AttnLabelConverter, Averager
from dataset import hierarchical_dataset, AlignCollate, Batch_Balanced_Dataset
from model import Model
from test import validation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'device: {device}')

device: cuda


In [ ]:
def train(opt):
    """ dataset preparation """
    if not opt.data_filtering_off:
        print('Filtering the images containing characters which are not in opt.character')
        print('Filtering the images whose label is longer than opt.batch_max_length')
        # see https://github.com/clovaai/deep-text-recognition-benchmark/blob/6593928855fb7abb999a99f428b3e4477d4ae356/dataset.py#L130

    opt.select_data = opt.select_data.split('-')
    opt.batch_ratio = opt.batch_ratio.split('-')
    train_dataset = Batch_Balanced_Dataset(opt)

    log = open(f'./saved_models/{opt.exp_name}/log_dataset.txt', 'a')
    AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD)
    valid_dataset, valid_dataset_log = hierarchical_dataset(root=opt.valid_data, opt=opt)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=opt.batch_size,
        shuffle=True,  # 'True' to check training progress with validation function.
        num_workers=int(opt.workers),
        collate_fn=AlignCollate_valid, pin_memory=True)
    log.write(valid_dataset_log)
    print('-' * 80)
    log.write('-' * 80 + '\n')
    log.close()
    
    """ model configuration """
    if 'CTC' in opt.Prediction:
        if opt.baiduCTC:
            converter = CTCLabelConverterForBaiduWarpctc(opt.character)
        else:
            converter = CTCLabelConverter(opt.character)
    else:
        converter = AttnLabelConverter(opt.character)
    opt.num_class = len(converter.character)

    if opt.rgb:
        opt.input_channel = 3
    model = Model(opt)
    print('model input parameters', opt.imgH, opt.imgW, opt.num_fiducial, opt.input_channel, opt.output_channel,
          opt.hidden_size, opt.num_class, opt.batch_max_length, opt.Transformation, opt.FeatureExtraction,
          opt.SequenceModeling, opt.Prediction)

    # weight initialization
    for name, param in model.named_parameters():
        if 'localization_fc2' in name:
            print(f'Skip {name} as it is already initialized')
            continue
        try:
            if 'bias' in name:
                init.constant_(param, 0.0)
            elif 'weight' in name:
                init.kaiming_normal_(param)
        except Exception as e:  # for batchnorm.
            if 'weight' in name:
                param.data.fill_(1)
            continue

    # data parallel for multi-GPU
    model = torch.nn.DataParallel(model).to(device)
    model.train()
    if opt.saved_model != '':
        print(f'loading pretrained model from {opt.saved_model}')
        if opt.FT:
            model.load_state_dict(torch.load(opt.saved_model), strict=False)
        else:
            model.load_state_dict(torch.load(opt.saved_model))
    print("Model:")
    print(model)

    """ setup loss """
    if 'CTC' in opt.Prediction:
        if opt.baiduCTC:
            # need to install warpctc. see our guideline.
            from warpctc_pytorch import CTCLoss 
            criterion = CTCLoss()
        else:
            criterion = torch.nn.CTCLoss(zero_infinity=True).to(device)
    else:
        criterion = torch.nn.CrossEntropyLoss(ignore_index=0).to(device)  # ignore [GO] token = ignore index 0
    # loss averager
    loss_avg = Averager()

    # filter that only require gradient decent
    filtered_parameters = []
    params_num = []
    for p in filter(lambda p: p.requires_grad, model.parameters()):
        filtered_parameters.append(p)
        params_num.append(np.prod(p.size()))
    print('Trainable params num : ', sum(params_num))
    # [print(name, p.numel()) for name, p in filter(lambda p: p[1].requires_grad, model.named_parameters())]

    # setup optimizer
    if opt.adam:
        optimizer = optim.Adam(filtered_parameters, lr=opt.lr, betas=(opt.beta1, 0.999))
    else:
        optimizer = optim.Adadelta(filtered_parameters, lr=opt.lr, rho=opt.rho, eps=opt.eps)
    print("Optimizer:")
    print(optimizer)

    """ final options """
    # print(opt)
    with open(f'./saved_models/{opt.exp_name}/opt.txt', 'a') as opt_file:
        opt_log = '------------ Options -------------\n'
        args = vars(opt)
        for k, v in args.items():
            opt_log += f'{str(k)}: {str(v)}\n'
        opt_log += '---------------------------------------\n'
        print(opt_log)
        opt_file.write(opt_log)

    """ start training """
    start_iter = 0
    if opt.saved_model != '':
        try:
            start_iter = int(opt.saved_model.split('_')[-1].split('.')[0])
            print(f'continue to train, start_iter: {start_iter}')
        except:
            pass

    start_time = time.time()
    best_accuracy = -1
    best_norm_ED = -1
    iteration = start_iter

    while(True):
        # train part
        image_tensors, labels = train_dataset.get_batch()
        image = image_tensors.to(device)
        text, length = converter.encode(labels, batch_max_length=opt.batch_max_length)
        batch_size = image.size(0)

        if 'CTC' in opt.Prediction:
            preds = model(image, text)
            preds_size = torch.IntTensor([preds.size(1)] * batch_size)
            if opt.baiduCTC:
                preds = preds.permute(1, 0, 2)  # to use CTCLoss format
                cost = criterion(preds, text, preds_size, length) / batch_size
            else:
                preds = preds.log_softmax(2).permute(1, 0, 2)
                cost = criterion(preds, text, preds_size, length)

        else:
            preds = model(image, text[:, :-1])  # align with Attention.forward
            target = text[:, 1:]  # without [GO] Symbol
            cost = criterion(preds.view(-1, preds.shape[-1]), target.contiguous().view(-1))

        model.zero_grad()
        cost.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), opt.grad_clip)  # gradient clipping with 5 (Default)
        optimizer.step()

        loss_avg.add(cost)

        # validation part
        if (iteration + 1) % opt.valInterval == 0 or iteration == 0: # To see training progress, we also conduct validation when 'iteration == 0' 
            elapsed_time = time.time() - start_time
            # for log
            with open(f'./saved_models/{opt.exp_name}/log_train.txt', 'a') as log:
                model.eval()
                with torch.no_grad():
                    valid_loss, current_accuracy, current_norm_ED, preds, confidence_score, labels, infer_time, length_of_data = validation(
                        model, criterion, valid_loader, converter, opt)
                model.train()

                # training loss and validation loss
                loss_log = f'[{iteration+1}/{opt.num_iter}] Train loss: {loss_avg.val():0.5f}, Valid loss: {valid_loss:0.5f}, Elapsed_time: {elapsed_time:0.5f}'
                loss_avg.reset()

                current_model_log = f'{"Current_accuracy":17s}: {current_accuracy:0.3f}, {"Current_norm_ED":17s}: {current_norm_ED:0.2f}'

                # keep best accuracy model (on valid dataset)
                if current_accuracy > best_accuracy:
                    best_accuracy = current_accuracy
                    torch.save(model.state_dict(), f'./saved_models/{opt.exp_name}/best_accuracy.pth')
                if current_norm_ED > best_norm_ED:
                    best_norm_ED = current_norm_ED
                    torch.save(model.state_dict(), f'./saved_models/{opt.exp_name}/best_norm_ED.pth')
                best_model_log = f'{"Best_accuracy":17s}: {best_accuracy:0.3f}, {"Best_norm_ED":17s}: {best_norm_ED:0.2f}'

                loss_model_log = f'{loss_log}\n{current_model_log}\n{best_model_log}'
                print(loss_model_log)
                log.write(loss_model_log + '\n')

                # show some predicted results
                dashed_line = '-' * 80
                head = f'{"Ground Truth":25s} | {"Prediction":25s} | Confidence Score & T/F'
                predicted_result_log = f'{dashed_line}\n{head}\n{dashed_line}\n'
                for gt, pred, confidence in zip(labels[:5], preds[:5], confidence_score[:5]):
                    if 'Attn' in opt.Prediction:
                        gt = gt[:gt.find('[s]')]
                        pred = pred[:pred.find('[s]')]

                    predicted_result_log += f'{gt:25s} | {pred:25s} | {confidence:0.4f}\t{str(pred == gt)}\n'
                predicted_result_log += f'{dashed_line}'
                print(predicted_result_log)
                log.write(predicted_result_log + '\n')

        # save model per 1e+5 iter.
        if (iteration + 1) % 1e+5 == 0:
            torch.save(
                model.state_dict(), f'./saved_models/{opt.exp_name}/iter_{iteration+1}.pth')

        if (iteration + 1) == opt.num_iter:
            print('end the training')
            sys.exit()
        iteration += 1

In [ ]:
'''
수정소요
parser.add_argument('--train_data', required=True, help='path to training dataset') --> parser.add_argument('--train_data', default='data_lmdb/train', help='path to training dataset')
parser.add_argument('--valid_data', required=True, help='path to validation dataset') --> parser.add_argument('--valid_data', default='data_lmdb/test', help='path to validation dataset')
parser.add_argument('--Transformation', type=str, required=True, help='Transformation stage. None|TPS') --> parser.add_argument('--Transformation', type=str, default='TPS', help='Transformation stage. None|TPS')
parser.add_argument('--FeatureExtraction', type=str, required=True, help='FeatureExtraction stage. VGG|RCNN|ResNet') --> parser.add_argument('--FeatureExtraction', type=str, default='ResNet', help='FeatureExtraction stage. VGG|RCNN|ResNet')
parser.add_argument('--SequenceModeling', type=str, required=True, help='SequenceModeling stage. None|BiLSTM') --> parser.add_argument('--SequenceModeling', type=str, default='BiLSTM', help='SequenceModeling stage. None|BiLSTM')
parser.add_argument('--Prediction', type=str, required=True, help='Prediction stage. CTC|Attn') --> parser.add_argument('--Prediction', type=str, default='CTC', help='Prediction stage. CTC|Attn')
parser.add_argument('--num_iter', type=int, default=300000, help='number of iterations to train for') --> parser.add_argument('--num_iter', type=int, default=30000, help='number of iterations to train for') # epoch
parser.add_argument('--valInterval', type=int, default=2000, help='Interval between each validation') --> parser.add_argument('--valInterval', type=int, default=200, help='Interval between each validation') # print_val_freq
'''




if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--exp_name', help='Where to store logs and models')
    parser.add_argument('--train_data', default='data_lmdb/train', help='path to training dataset')
    parser.add_argument('--valid_data', default='data_lmdb/test', help='path to validation dataset')
    parser.add_argument('--manualSeed', type=int, default=3333, help='for random seed setting')
    parser.add_argument('--workers', type=int, help='number of data loading workers', default=4)
    parser.add_argument('--batch_size', type=int, default=192, help='input batch size')
    parser.add_argument('--num_iter', type=int, default=30000, help='number of iterations to train for')
    parser.add_argument('--valInterval', type=int, default=200, help='Interval between each validation')
    parser.add_argument('--saved_model', default='', help="path to model to continue training")
    parser.add_argument('--FT', action='store_true', help='whether to do fine-tuning')
    parser.add_argument('--adam', action='store_true', help='Whether to use adam (default is Adadelta)')
    parser.add_argument('--lr', type=float, default=1, help='learning rate, default=1.0 for Adadelta')
    parser.add_argument('--beta1', type=float, default=0.9, help='beta1 for adam. default=0.9')
    parser.add_argument('--rho', type=float, default=0.95, help='decay rate rho for Adadelta. default=0.95')
    parser.add_argument('--eps', type=float, default=1e-8, help='eps for Adadelta. default=1e-8')
    parser.add_argument('--grad_clip', type=float, default=5, help='gradient clipping value. default=5')
    parser.add_argument('--baiduCTC', action='store_true', help='for data_filtering_off mode')
    """ Data processing """
    parser.add_argument('--select_data', type=str, default='/',
                        help='select training data (default is MJ-ST, which means MJ and ST used as training data)')
    parser.add_argument('--batch_ratio', type=str, default='1',
                        help='assign ratio for each selected data in the batch')
    parser.add_argument('--total_data_usage_ratio', type=str, default='1.0',
                        help='total data usage ratio, this ratio is multiplied to total number of data.')
    parser.add_argument('--batch_max_length', type=int, default=25, help='maximum-label-length')
    parser.add_argument('--imgH', type=int, default=32, help='the height of the input image')
    parser.add_argument('--imgW', type=int, default=100, help='the width of the input image')
    parser.add_argument('--rgb', action='store_true', help='use rgb input')
    parser.add_argument('--character', type=str,
                        default=' !"#%&\'()*+,-./0123456789:<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz~가각간갇갈갉감갑값갓갔강갖같갚갛개객갠갤갬갭갯갱갸갼걀걍거걱건걷걸검겁것겅겉게겐겔겜겟겠겡겨격겪견결겸겹겼경곁계고곡곤곧골곰곱곳공곶과곽관괄괌광괘괜괭괴괸굄굉교굣구국군굳굴굶굼굽굿궁궂궈권궐궤귀귄귐귓규균귤그극근글금급긋긍기긱긴긷길김깁깃깅깊까깍깎깐깔깜깝깡깥깨깬깻깽꺄꺼꺽꺾껀껄껌껍껏껑께껜껨껴껸꼈꼐꼬꼭꼰꼴꼼꼽꽁꽂꽃꽈꽉꽐꽤꽥꽹꾀꾸꾹꾼꿀꿇꿈꿉꿍꿔꿨꿩꿰뀀뀌뀐뀔뀝뀨끄끈끊끌끎끓끔끗끙끝끼끽낀낄낌낑나낙낚난날낡남납낫났낭낮낯낱낳내낵낸낼냄냅냇냈냉냐냘냠냥너넉넋넌널넓넘넙넛넜넝넣네넥넨넬넴넵넷넹녀녁년념녔녕녘녜노녹논놀놈놉놋농높놓놔뇌뇨뇽누눅눈눌눔눕눗눙눠뉘뉜뉴늄느늑는늘늙늠능늦늪늬니닉닌닐님닙닛닝닢다닥닦단닫달닭닮담답닷당닻닿대댁댄댈댐댑댓댔댕댜더덕던덜덟덤덥덧덩덫덮데덱덴델뎀뎁뎃뎅뎌뎐도독돈돋돌돔돕돗동돛돝돼됐되된될됨됩두둑둔둘둠둡둣둥둬뒀뒤뒷뒹듀듄듈드득든듣들듦듬듭듯등듸디딕딘딛딜딤딥딧딩딪따딱딴딸땀땁땅땋때땍땐땔땜땠땡떠떡떤떨떳떴떵떻떼떽뗌뗏뗐뗑또똑똔똘똥뙈뚜뚝뚠뚤뚫뚱뛰뛴뛸뜀뜨뜩뜬뜯뜰뜸뜻띄띠띤띨띵라락란랄람랍랏랐랑랗래랙랜랠램랩랫랬랭랴략랸량러럭런럴럼럽럿렀렁렇레렉렌렐렘렙렛렝려력련렬렴렵렷렸령례로록론롤롬롭롯롱롸뢰료룔룡루룩룬룰룸룹룻룽뤄뤠뤼륀륄류륙륜률륨륫륭르륵른를름릅릇릉릎리릭린릴림립릿링마막만많맏말맑맘맙맛망맞맡맣매맥맨맴맵맷맹맺먀머먹먼멀멈멋멍메멕멘멜멤멥멧멩며멱면멸명몇모목몫몬몰몸몹못몽뫼묘무묵묶문묻물묾뭄뭇뭉뭍뭐뭔뭘뭣뮈뮌뮐뮤뮨뮬므믄믈믐미믹민믿밀밈밋밌밍및밑바박밖반받발밝밟밤밥밧방밭배백밴밸뱀뱁뱃뱅뱉뱍뱐버벅번벌범법벗벙벚베벡벤벧벨벰벱벳벵벼벽변별볍볏병볕보복볶본볼봄봅봇봉봐봤봬뵈뵙부북분불붉붐붑붓붕붙붚뷔뷜뷰브븍븐블븜븟비빅빈빌빔빕빗빙빚빛빠빡빤빨빰빱빳빵빻빼빽뺀뺄뺍뺑뺨뻐뻑뻔뻗뻘뻣뻤뻥뻬뼈뼘뼛뼝뽀뽁뽄뽈뽐뽑뽕뾰뿅뿌뿍뿐뿔뿜뿡쁘쁜쁠쁨삐삑삘삠사삭산살삶삼삽삿샀상새색샌샐샘샙샛생샤샥샨샬샴샵샷샹섀서석섞선설섧섬섭섯섰성섶세섹센셀셈셉셋셍셔션셜셧셨셩셰셴셸소속손솔솜솝솟송솥솨쇄쇠쇳쇼숀숄숍숏숑수숙순숟술숨숩숫숭숯숱숲숴쉐쉑쉔쉘쉬쉭쉰쉴쉼쉽슁슈슉슐슘슝스슥슨슬슴습슷승시식신싣실싫심십싯싱싶싸싹싼쌀쌈쌉쌌쌍쌓쌔쌕쌘쌤쌩썅써썩썬썰썸썹썼썽쎄쎈쎌쏀쏘쏙쏜쏟쏠쏨쏭쏴쐐쑈쑤쑥쑨쓰쓱쓴쓸씀씁씌씨씩씬씰씸씹씻씽아악안앉않알앎앓암압앗았앙앞애액앤앨앰앱앳앵야약얀얄얇얌얍얏양얕얗얘얜어억언얹얻얼얽엄업없엇었엉엌엎에엑엔엘엠엡엣엥여역엮연열염엽엿였영옅옆예옌옐옙옛옜오옥온올옮옳옴옵옷옹옻와왁완왈왑왓왔왕왜왠왱외왼요욕욘욜욤욥욧용우욱운울움웁웃웅워웍원월웜웠웨웩웬웰웸웹위윅윈윌윔윗윙유육윤율윰융윷으윽은을읊음읍응읔읖의이익인일읽잃임입잇있잉잊잎자작잔잖잗잘잠잡잣잤장잦재잭잰잼잽잿쟁쟈쟉쟌쟝저적전절젊점접젓정젖제젝젠젤젬젯져젼졌졔조족존졸좀좁종좇좋좌죄죠죤주죽준줄줌줍줏중줘줬쥐쥑쥔쥘쥬쥰쥴즈즉즌즐즘즙증지직진짇질짊짐집짓징짖짙짚짜짝짠짤짧짬짭짱째짼쨈쨋쨌쨍쩌쩍쩐쩔쩜쩝쩡쩨쪄쪘쪼쪽쫀쫄쫌쫑쫓쫘쭈쭉쭌쭘쭝쮸쯔쯤찌찍찐찔찜찝찡찢찧차착찬찮찰참찹찻창찾채책챈챌챔챕챗챙챠챤챨처척천철첨첩첫청체첸첼쳄쳅쳇쳐쳔쳤쳬초촉촌촘촛총촨촬최쵸추축춘출춤춥춧충춰췄췌취췬츄츈츠측츨츰층치칙친칠칡침칩칫칭카칵칸칼캄캅캇캉캐캔캘캠캡캣캥캬커컨컬컴컵컷컸컹케켄켈켐켑켓켜켠켤켰코콕콘콜콤콥콧콩콰콴콸쾅쾌쾰쿄쿠쿡쿤쿨쿰쿱쿳쿵쿼퀀퀄퀘퀴퀵퀸퀼큅큐큘크큰클큼킁키킥킨킬킴킵킷킹타탁탄탈탉탐탑탓탕태택탠탤탬탭탯탱터턱턴털텀텁텃텅테텍텐텔템텝텟텨텬톈토톡톤톨톰톱톳통톺퇘퇴툇투툭툰툴툼툽퉁튀튄튈튑튜튠튤튬트특튼틀틈틉틋틔티틱틴틸팀팁팃팅파팍팎판팔팜팝팟팠팡팥패팩팬팰팸팹팻팽퍼펀펄펌펍펏펑페펙펜펠펨펫펭펴편펼평폐포폭폰폴폼폿퐁푀표푸푹푼풀풂품풉풋풍퓌퓨퓰퓸프픈플픔피픽핀필핌핍핏핑하학한할함합핫항해핵핸핼햄햅햇했행햐향허헉헌헐험헙헛헝헤헥헨헬헴헵헷헹혀혁현혈혐협혔형혜호혹혼홀홈홉홋홍화확환활홧황횃회획횟횡효후훅훈훌훔훗훙훠훤훨훼휀휄휘휙휜휠휩휴휼흉흐흑흔흘흙흠흡흥흩희흰히힉힌힐힘힙힛힝', help='character label')
    parser.add_argument('--sensitive', action='store_true', help='for sensitive character mode')
    parser.add_argument('--PAD', action='store_true', help='whether to keep ratio then pad for image resize')
    parser.add_argument('--data_filtering_off', action='store_true', help='for data_filtering_off mode')
    """ Model Architecture """
    parser.add_argument('--Transformation', type=str, default='TPS', help='Transformation stage. None|TPS')
    parser.add_argument('--FeatureExtraction', type=str, default='ResNet', help='FeatureExtraction stage. VGG|RCNN|ResNet')
    parser.add_argument('--SequenceModeling', type=str, default='BiLSTM', help='SequenceModeling stage. None|BiLSTM')
    parser.add_argument('--Prediction', type=str, default='CTC', help='Prediction stage. CTC|Attn')
    parser.add_argument('--num_fiducial', type=int, default=20, help='number of fiducial points of TPS-STN')
    parser.add_argument('--input_channel', type=int, default=1,
                        help='the number of input channel of Feature extractor')
    parser.add_argument('--output_channel', type=int, default=512,
                        help='the number of output channel of Feature extractor')
    parser.add_argument('--hidden_size', type=int, default=256, help='the size of the LSTM hidden state')

    opt = parser.parse_args(args=[])

    if not opt.exp_name:
        opt.exp_name = f'{opt.Transformation}-{opt.FeatureExtraction}-{opt.SequenceModeling}-{opt.Prediction}'
        opt.exp_name += f'-Seed{opt.manualSeed}'
        # print(opt.exp_name)

    os.makedirs(f'./saved_models/{opt.exp_name}', exist_ok=True)

    """ vocab / character number configuration """
    if opt.sensitive:
        # opt.character += 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
        opt.character = string.printable[:-6]  # same with ASTER setting (use 94 char).

    """ Seed and GPU setting """
    # print("Random Seed: ", opt.manualSeed)
    random.seed(opt.manualSeed)
    np.random.seed(opt.manualSeed)
    torch.manual_seed(opt.manualSeed)
    torch.cuda.manual_seed(opt.manualSeed)

    cudnn.benchmark = True
    cudnn.deterministic = True
    opt.num_gpu = torch.cuda.device_count()
    # print('device count', opt.num_gpu)
    if opt.num_gpu > 1:
        print('------ Use multi-GPU setting ------')
        print('if you stuck too long time with multi-GPU setting, try to set --workers 0')
        # check multi-GPU issue https://github.com/clovaai/deep-text-recognition-benchmark/issues/1
        opt.workers = opt.workers * opt.num_gpu
        opt.batch_size = opt.batch_size * opt.num_gpu

        """ previous version
        print('To equlize batch stats to 1-GPU setting, the batch_size is multiplied with num_gpu and multiplied batch_size is ', opt.batch_size)
        opt.batch_size = opt.batch_size * opt.num_gpu
        print('To equalize the number of epochs to 1-GPU setting, num_iter is divided with num_gpu by default.')
        If you dont care about it, just commnet out these line.)
        opt.num_iter = int(opt.num_iter / opt.num_gpu)
        """

    train(opt)

Filtering the images containing characters which are not in opt.character
Filtering the images whose label is longer than opt.batch_max_length
--------------------------------------------------------------------------------
dataset_root: data_lmdb/train
opt.select_data: ['/']
opt.batch_ratio: ['1']
--------------------------------------------------------------------------------
dataset_root:    data_lmdb/train	 dataset: /
sub-directory:	/.	 num samples: 725470
num total samples of /: 725470 x 1.0 (total_data_usage_ratio) = 725470
num samples of / per batch: 192 x 1.0 (batch_ratio) = 192


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


--------------------------------------------------------------------------------
Total_batch_size: 192 = 192
--------------------------------------------------------------------------------
dataset_root:    data_lmdb/test	 dataset: /
sub-directory:	/.	 num samples: 91477
--------------------------------------------------------------------------------
model input parameters 32 100 20 1 512 256 1784 25 TPS ResNet BiLSTM CTC
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.LocalizationNetwork.localization_fc2.bias as it is already initialized
Model:
DataParallel(
  (module): Model(
    (Transformation): TPS_SpatialTransformerNetwork(
      (LocalizationNetwork): LocalizationNetwork(
        (conv): Sequential(
          (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1/30000] Train loss: 52.92694, Valid loss: 53.65083, Elapsed_time: 3.06150
Current_accuracy : 0.000, Current_norm_ED  : 0.00
Best_accuracy    : 0.000, Best_norm_ED     : 0.00
--------------------------------------------------------------------------------
Ground Truth              | Prediction                | Confidence Score & T/F
--------------------------------------------------------------------------------
문경시의회 제251회 임시회           | 깃것청줬깃팰토팰토빔토빔토             | 0.0000	False
지질연, 수소경제 실현하는            | 깃것줬팰토                     | 0.0000	False
상담클리                      | 깃띤청취청취뿐토                  | 0.0000	False
행복                        | 것뿅깃토                      | 0.0000	False
독서실                       | 팰청잘깃청깃청깃찡깃번토              | 0.0000	False
--------------------------------------------------------------------------------
[200/30000] Train loss: 6.96636, Valid loss: 5.98951, Elapsed_time: 521.54742
Current_accuracy : 0.016, Current_norm_ED  : 0.03
Best_accuracy    : 0.016, Bes

SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
